In [16]:
import pandas as pd
import re
import yfinance as yf
pd.options.display.float_format = "{:,.2f}".format

In [17]:
def get_current_stock_price(ticker):
    stock_info = yf.Ticker(ticker).info
    # stock_info.keys() for other properties you can explore
    market_price = stock_info['regularMarketPrice']
    previous_close_price = stock_info['regularMarketPreviousClose']
    print(ticker + ' stock market price ', market_price)
    print(ticker + ' stock previous close price ', previous_close_price)
    return market_price

### **Current key ratios** (comparing against competition)

In [18]:
tickers = ['NVDA', 'INTC', 'AMD']
current_key_ratios_df = pd.DataFrame([])
for ticker in tickers:
    # ticker = 'DOCN'
    path = './datasets/' + ticker + '/'
    filename = ticker + '-current-key-ratios.csv'
    filepath = path + filename
    # print(filepath)
    df = pd.read_csv(filepath, encoding='utf-8', index_col=0)
    if len(current_key_ratios_df.index) == 0:
        current_key_ratios_df = df.copy()
    else:
        current_key_ratios_df = current_key_ratios_df.join(df)

# current_key_ratios_df

In [20]:
ticker = 'INTC'
current_stock_price = get_current_stock_price(ticker)
ordered_tickets = [ticker] + [x for x in tickers if x != ticker]
print(ordered_tickets)
current_key_ratios_df = current_key_ratios_df.loc[:, ordered_tickets]
current_key_ratios_df['Promedio'] = current_key_ratios_df.mean(axis=1).round(2)
current_key_ratios_df.loc['Price/Earnings':'Price/Book', 'Precio'] = current_stock_price
current_key_ratios_df['Valor'] = (current_key_ratios_df.loc[:, 'Promedio'] /current_key_ratios_df.iloc[:, 0] * current_key_ratios_df.loc[:, 'Precio']).round(2)
current_key_ratios_df.fillna('-', inplace=True)
current_key_ratios_df

INTC stock market price  48.44
INTC stock previous close price  47.58
['INTC', 'NVDA', 'AMD']


,INTC,NVDA,AMD,Promedio,Precio,Valor
Current Ratio,2.10,7.14,2.02,3.75,-,-
Quick Ratio,1.38,6.44,1.49,3.10,-,-
Cash Ratio,1.03,5.34,0.85,2.41,-,-
Debt/Equity,0.35,0.49,0.05,0.30,-,-
Inventory Turnover,3.66,4.64,5.07,4.46,-,-
Days Inventory,99.54,78.73,71.97,83.41,-,-
Asset Turnover,0.49,0.72,1.54,0.92,-,-
Return on Equity %,22.52,41.94,47.43,37.30,-,-
Net Margin %,25.14,33.81,19.24,26.06,-,-
Price/Earnings,9.80,74.80,44.50,43.03,48.44,212.69


### **Historico**

In [23]:
path = './datasets/' + ticker + '/'
filename = ticker + '-current-past-valuation-ratios.csv'
filepath = path + filename
# print(filepath)
df = pd.read_csv(filepath, encoding='utf-8', index_col=0)
df['Price'] = current_stock_price
df['Valor'] = (df.iloc[:, 1] / df.iloc[:, 0] * df.iloc[:, -1]).round(2)
df

,INTC,5 years,Price,Valor
Price/Earnings,9.80,14.10,48.44,69.69
Price/Cash Flow,6.50,7.30,48.44,54.40
Price/Sales,2.50,3.20,48.44,62.00
Price/Book,2.00,2.90,48.44,70.24


In [22]:
df1 = pd.DataFrame([], index=[0])
df1['Historico'] = df['Valor'].mean()
df1['Intrinseco por Industria'] = current_key_ratios_df.loc['Price/Earnings':'Price/Book', 'Valor'].mean()
df1['Valor Final'] = (df1['Historico'] + df1['Intrinseco por Industria']) / 2
df1

,Historico,Intrinseco por Industria,Valor Final
0,64.08,279.94,172.01
